# AppGPS


### What makes people to like or dislike an App in the category communication of Google Play Store?


### What characterize the description of best ranket Apps


In [1]:
import numpy as np
import pandas as pd
# For visualizations
import matplotlib.pyplot as plt
from matplotlib.colors import BASE_COLORS

%matplotlib inline


import seaborn as sns
sns.set_theme(style="ticks", color_codes=True)

# For regular expressions
import re
# For handling string
import string
# For performing mathematical operations
import math
# For perfomance tracking 
from tqdm.notebook import trange, tqdm


# For tokenazing the text with Python
import spacy
# For Document Term Matrix
from sklearn.feature_extraction.text import CountVectorizer 

import nltk
# For counting unique words
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

import pickle

In [99]:
data = pd.read_csv("lematized_data.csv") 

In [100]:
# delete all the rows with score 3 to avoid neutral values and train the test with negative and positives

data = data[data.score != 3]

In [101]:
# create a label for sentiment analysis
data['label'] = data['score'].apply(lambda x: 'Positive' if x >= 4 else 'Negative')

#data['label'] = data['score'].apply(lambda x: 'Positive' if x > 3 else('Neutral' if x == 3  else 'Negative'))


In [102]:
#create a target
data['target'] = data['score'].apply(lambda x: 1 if x >= 4 else 0)

In [103]:

data_clean = data.dropna(how='any', subset=['lemmatized'], axis=0)

In [104]:
data_clean.head()

,score,thumbsupcount,at,appid,content,lemmatized,label,target
0,1,881,2021-03-05 10:53:50,com.android.chrome,just got the update and now chrome has just st...,get update chrome stop work reason try clear c...,Negative,0
1,1,570,2021-03-06 20:56:06,com.android.chrome,i used to love this but the grouped tabs are i...,love group tab illogical add extra click want ...,Negative,0
2,1,31,2021-03-05 19:19:02,com.android.chrome,i was super happy with chrome for android this...,super happy chrome android change immediately ...,Negative,0
3,1,148,2021-03-04 09:21:44,com.android.chrome,when i enter google on my phone and enter a ce...,enter google phone enter certain site switch g...,Negative,0
4,1,41,2021-03-05 13:20:29,com.android.chrome,the new update is total bs the tab layout is t...,new update total bs tab layout totally mess ch...,Negative,0


In [105]:
data_clean.shape

(31387, 8)

# Sentiment Analysis with TFIDF and Random Forest


In [106]:
# defining X

features = data_clean.drop(columns= ["appid", "label", "at", "content", "thumbsupcount", "score", "target"], axis=1)

In [107]:
features

,lemmatized
0,get update chrome stop work reason try clear c...
1,love group tab illogical add extra click want ...
2,super happy chrome android change immediately ...
3,enter google phone enter certain site switch g...
4,new update total bs tab layout totally mess ch...
...,...
46109,good
46110,good fun
46111,noyon
46112,nice free app


In [108]:
labels = data_clean["target"]

In [109]:
labels.value_counts()

0    16220
1    15167
Name: target, dtype: int64

### Apply Random Forest
source: https://www.kaggle.com/onadegibert/sentiment-analysis-with-tfidf-and-random-forest

In [110]:
from sklearn.model_selection import train_test_split

#features = imdb.drop("label",axis=1)
#labels = imdb["label"]

X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size = 0.90, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size = 0.5, random_state=42)
#X_val, X_test_val, y_val, y_test_val = train_test_split(X_test,y_test,test_size = 0.5, random_state=42)

print("Data distribution:\n- Train: {} \n- Test: {}".format(len(y_train), len(y_test)))

Data distribution:
- Train: 1569 
- Test: 1569


In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# clean again the text

def clean(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    lower = [word.lower() for word in tokens]
    no_stopwords = [word for word in lower if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    lemm_text = [wn.lemmatize(word) for word in no_alpha]
    clean_text = lemm_text
    return clean_text

# Vectorize

def vectorize(data,tfidf_vect_fit):
    X_tfidf = tfidf_vect_fit.transform(data)
    words = tfidf_vect_fit.get_feature_names()
    X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
    X_tfidf_df.columns = words
    return(X_tfidf_df)

# convert the word in numbers
tfidf_vect = TfidfVectorizer(analyzer=clean)
tfidf_vect_fit=tfidf_vect.fit(X_train['lemmatized'])
X_train=vectorize(X_train['lemmatized'],tfidf_vect_fit)


# running the model
rf = RandomForestClassifier(random_state=0)
scores = cross_val_score(rf,X_train,y_train.values.ravel(),cv=5)

print(scores)
scores.mean()

[0.72611465 0.70382166 0.75159236 0.6910828  0.73801917]


0.7221261268594452

In [112]:
X_train.shape


(1569, 3231)

In [113]:
# Store the model as picklefile
import pickle
with open('model_trained_apss_final', 'wb') as picklefile:
    pickle.dump(rf,picklefile)
